In [45]:
import mesa
import numpy as np
from numpy import random
random.seed(189)

In [2]:
TIMESTEP = 15 # minutes = 1 tick
STATES = ['idle', 'charging', 'pickup', 'dropoff', 'going to recharge'] # car states
SPEED = 8 # miles per tick
CHARGING = 80 # kW outputted per hour by charging station
FUEL = 1 / 5 # kWh used to drive 1 mile

In [286]:
def manhattan_distance(pos1, pos2):
    # MANHATTAN DISTANCE from POS1 to POS2
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def random_pos(x, y):
    # RANDOM POSITION on the X by Y board
    return (random.randint(x - 1), random.randint(y - 1))

x, y = 25, 25
generate_pos = lambda : random_pos(x, y)

In [304]:
class Car(mesa.Agent):
    def __init__(self, id, model, pos, capacity = 80, battery = 1):
        # INITIALIZE CAR agent at position POS in MODEL, with battery capacity CAPACITY in kWh,
        # and battery level BATTERY
        super().__init__(id, model)
        model.place(self, pos)
        self.battery = battery
        self.capacity = capacity
        self.state = 'idle'
        self.dest = None
        self.pickup = None
        self.charging_time = 0 # minutes to finish charging to 100
        self.profit = 0

    def go_charge(self):
        self.state = 'going to recharge'
        self.dest = self.find_closest_station()

    def start_new_itinerary(self, pickup, dropoff):

        dist_to_pickup = manhattan_distance(self.pos, pickup)
        dist_to_dest = manhattan_distance(pickup, dropoff)
        closest_station = self.find_closest_station(dropoff)
        dist_to_charging = manhattan_distance(dropoff, closest_station)
        total_dist = dist_to_pickup + dist_to_dest + dist_to_charging
        battery_needed = total_dist * FUEL

        if self.battery * self.capacity >= battery_needed:
            self.state = 'pickup'
            self.pickup = pickup
            self.dest = dropoff
            self.profit += total_dist
        else:
            self.go_charge()

    def step(self):
        if self.state == 'charging':
            self.battery += (80 * TIMESTEP / 60) / self.capacity
        elif self.state == 'idle':
            self.start_new_itinerary(generate_pos(), generate_pos())
        else:
            dest = self.pickup if self.pickup else self.dest
            dx = abs(self.pos[0] - dest[0])
            dy = abs(self.pos[1] - dest[1])
            if SPEED >= dx + dy:
                self.move(dx, dy)
                self.update_state()
            elif SPEED <= dx:
                self.move(SPEED, 0)
            elif SPEED <= dy:
                self.move(0, SPEED)
            else:
                remainder = SPEED - dx
                assert remainder > 0, "remainder not greater than 0?!"
                self.move(dx, remainder)
                
    def update_state(self):
        state = self.state
        match state:
            case 'pickup':
                self.state = 'dropoff'
                self.pickup = None
            case 'going to recharge':
                self.state = 'charging'
            case 'dropoff':
                self.state = 'idle'
            case 'charging':
                self.state = 'idle'

    def move(self, dx, dy):
        # MOVE DX units in the x axis towards destination, DY units in y axis towards destination
        # updates battery accordingly. DY, DX positive for simplicity.
        assert dx >= 0 and dy >= 0, "DX, DY not >= 0!"
        self.battery -= (dy + dx) * FUEL / self.capacity
        self.profit += dy + dx
        dest = self.pickup if self.pickup else self.dest
        dx *= -1 if self.pos[0] > dest[0] else 1
        dy *= -1 if self.pos[1] > dest[1] else 1
        new_pos = (self.pos[0] + dx, self.pos[1] + dy)
        self.model.grid.move_agent(self, new_pos)

    def find_closest_station(self, start = None):
        stations = list(self.model.charging_stations)
        pos = start if start else self.pos
        distances = [(manhattan_distance(pos, s), s) for s in stations]
        distances.sort(key = lambda x: x[0])
        return distances[0][1]

class Charger(mesa.Agent):
    def __init__(self, id, model, pos, capacity = np.inf, rate = 1,):
        super().__init__(id, model)
        model.grid.place_agent(self, pos)
        model.charging_stations[pos] = self
        self.capacity = capacity
        self.rate = rate

class City(mesa.Model):
    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.grid = mesa.space.MultiGrid(rows, cols, False)
        self.schedule = mesa.time.BaseScheduler(self)
        self.charging_stations = {}
    
    def place(self, car, pos):
        # Places a car on the grid
        self.grid.place_agent(car, pos)
        self.schedule.add(car)
    
    def step(self):
        self.schedule.step()
    
    def print(self):
        header = list(range(self.cols))
        print('   ' + str(header.pop(0)) + ' ', end = '')
        [print(f' {x} ' if x < 10 else f'{x} ', end='') for x in header]
        print()
        for c in range(self.cols):
            s = str(c) + (' ' if c < 10 else '')
            for r in range(self.rows):
                agent = self.grid[r][c]
                if agent:
                    s += ' C ' if type(agent[0]) is Car else ' Ch'
                else:
                    s += ' . '
            print(s)


In [307]:
random.seed(189)
num_cars = 1
sf = City(x, y)
charging_stations = [(18, 15), (15, 8), (21, 8), (9, 17), (17, 10)]
cars = []
for i in range(num_cars):
    pos = (0, 0) # generate_pos()
    cars.append(Car(i, sf, pos))
for pos in charging_stations:
    Charger(num_cars + i, sf, pos)

car = cars[0]
for i in range(100000):
    sf.step()
car.profit
car.battery

24982.02